## This notebook plots snowpit stratigraphy

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import glob
from matplotlib.lines import Line2D
from datetime import datetime, timedelta
from scipy.stats import linregress
import scipy.io as sio
from copy import deepcopy
from aesop import relabel_pits

In [ ]:
matfile = sio.loadmat('../Data/ground_obs/TVC2018_trenches.mat',struct_as_record=False, squeeze_me=True)
trenchdata = matfile['TVC']

In [ ]:
# Make a list of profiles i.e. separate measurement locations
profile_list = dir(trenchdata)[0:36] # don't need other attributes of data, just profile names
# Profile A03E has thickest ice lens: 1cm but no WS layer
# Remove lake ice snowpits
profile_list = [s for s in profile_list if 'A1' not in s]

In [ ]:
# Re-order so Met Station is grouped with other A04 plots
profile_list = profile_list[:12] + ['MetS'] + profile_list[12:-1]

In [ ]:
# Relabel pits - reviewer suggestion
new_profile_list = relabel_pits(profile_list)

In [ ]:
# Extract layer thickness to make stratigraphy profile
layer_thickness = []
ice_loc = []
for p in profile_list:
    profile = eval('trenchdata' + '.' + p)
    layer_thickness.append(profile.layer_thickness.layer_thickness) # still in cm
    ice_loc.append(profile.ice_lens.ice_lens_height_top)
    

layer_thickness = np.asarray(layer_thickness)

# Deal with problematic / null ice lenses
ice_loc[7] = 19 # A04C1
ice_loc[17] = 39 # A05W
ice_loc[23] = np.nan # There is no ice lens recorded for A08C
ice_loc = np.asarray(ice_loc)

In [ ]:
# Stacked bar chart of layer thicknesses
width=0.7
ind = np.arange(len(new_profile_list))

fig = plt.figure()
ax = fig.gca()

p1 = ax.bar(ind, layer_thickness[:,2], width)
p2 = ax.bar(ind, layer_thickness[:,1], width, bottom=layer_thickness[:,2])
p3 = ax.bar(ind, layer_thickness[:,0], width, color='grey',
             bottom=np.nansum(np.dstack((layer_thickness[:,1],layer_thickness[:,2])),2).flatten())
p4 = ax.plot(ind, ice_loc, 'k.')

plt.ylabel('Height (cm)')
plt.xticks(ind, new_profile_list, rotation=90)
plt.legend((p1[0], p2[0], p3[0], p4[0]), ('DH', 'WS', 'SS', 'ice lens'), loc='upper right')

plt.tight_layout()
plt.show()
